In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pickle
from datetime import datetime

### Read data from SQL database

In [2]:
# access to sql database
dbname = 'patent_db'
username = 'jy'
pswd = 'jy'

engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))

In [4]:
# reading from sql database
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [23]:
# read data from 2004-2007
years = np.arange(2004, 2008)

# dataframe to store the results
patents = pd.DataFrame()

# import the numerical features from each table
for year in years:
    # query:
    sql_query = """
    SELECT classification, num_applications, 
        num_patent_citations, num_nonpatent_citations, 
        num_claims, num_similar_doc, num_authors,
        "publication date", id, title, payment_times
        FROM patents_%s;
    """ %str(year)

    results = pd.read_sql_query(sql_query,con)
    
    # combine the datasets from different years
    patents = pd.concat([patents, results], axis = 0)

In [8]:
# check dimension of the data
patents.shape

(12033, 11)

### Remove patents missing classification information
There are only a small number of patents missing the classification information. These patents may fall into a different class which is not encoded by the system. 

In [9]:
# check the number of patents missing classification data
missing_class_index = (patents['classification'].values == 'NA')
print "Patents missing classification: ", np.sum(missing_class_index)

Patents missing classification:  3


In [33]:
### remove the patents missing classification information
# reassign patent index
patents.index = range(len(patents.index))
# drop the rows
patents =  patents.drop(patents.index[missing_class_index])
patents.shape

(12030, 11)

### Clean the data
#### Extract patent publication year

In [11]:
# parse the publication date to find publication year
publication_year = []

for i in range(patents.shape[0]):
    date = datetime.strptime(patents['publication date'].values[0], '%Y-%m-%d')
    publication_year.append(date.year)

# create a dataframe to store the year together with the patent id
cleaned_data = pd.DataFrame({
    'id' : patents['id'],
    'publication_year' : publication_year
})

#### One-hot-encode classification
The first column is dropped to avoid multicollinearity.

In [12]:
# one-hot-encoding classifications
one_hot_class = pd.get_dummies(patents['classification'], drop_first=True)
one_hot_class.shape

(12030, 7)

In [13]:
# add the classification predictors to the features
cleaned_data = pd.concat([cleaned_data, one_hot_class], axis = 1)

#### Add quantitative predictors

In [16]:
# quantitative variable
quant_cols = patents.drop(['classification', 'id', 'publication date'], axis = 1)
quant_cols.shape

(12030, 7)

In [17]:
# check missing values in the quant columns
# there is no missing data, which is consistent with the data collection method
for i in range(quant_cols.shape[1]):
    percent_na = np.mean(np.isnan(quant_cols.iloc[:, i].values))
    
    print quant_cols.columns[i], percent_na

num_applications 0.0
num_patent_citations 0.0
num_nonpatent_citations 0.0
num_claims 0.0
num_similar_doc 0.0
num_authors 0.0
payment_times 0.0


In [18]:
# combine the quant predictors to the features
cleaned_data = pd.concat([cleaned_data, quant_cols], axis = 1)
cleaned_data.shape

(12030, 16)

### Save the cleaned non-text features

In [19]:
# double check the features
cleaned_data.head()

,id,publication_year,B,C,D,E,F,G,H,num_applications,num_patent_citations,num_nonpatent_citations,num_claims,num_similar_doc,num_authors,payment_times
0,US6699658B1,2004,0,1,0,0,0,0,0,5,28,34,42,1,4,3
1,US6699724B1,2004,0,0,0,0,0,1,0,32,47,44,25,0,4,3
2,US6690816B2,2004,0,0,0,0,0,1,0,8,9,0,32,1,4,1
3,US6711436B1,2004,0,0,0,0,0,0,0,4,105,109,45,7,1,3
4,US6711432B1,2004,0,0,0,0,0,0,0,7,15,114,44,3,4,3


In [15]:
# pickle dump the data for later use
pickle.dump(cleaned_data, open("patent_data/nontext_features.p", "wb"))

In [16]:
# also save a csv
cleaned_data.to_csv("patent_data/nontext_features.csv", index = False)